<h2>Note:</h2>

This notebook contains optional part that is not related to Mozgalo competition.

Notebook applies our trained model on a harder dataset with cca 30000 images and roughly 256 meaningful categories of objects.

In [1]:
import sklearn
from sklearn import decomposition
import torchfile
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np
import os
from shutil import copyfile
import binascii
from sklearn.mixture import GaussianMixture
import random
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm
import warnings
warnings.filterwarnings('ignore')

In [2]:
# since dataset is much bigger it needed to be split in chunks
dataset_chunk_0 = torchfile.load('256_set/featuresChunk0.t7')
dataset_chunk_1 = torchfile.load('256_set/featuresChunk1.t7')
dataset_chunk_2 = torchfile.load('256_set/featuresChunk2.t7')
dataset_chunk_3 = torchfile.load('256_set/featuresChunk3.t7')
dataset_chunk_4 = torchfile.load('256_set/featuresChunk4.t7')
dataset_chunk_5 = torchfile.load('256_set/featuresChunk5.t7')
dataset_chunk_6 = torchfile.load('256_set/featuresChunk6.t7')

In [3]:
dataset_list = []
dataset_list.append(dataset_chunk_0)
dataset_list.append(dataset_chunk_1)
dataset_list.append(dataset_chunk_2)
dataset_list.append(dataset_chunk_3)
dataset_list.append(dataset_chunk_4)
dataset_list.append(dataset_chunk_5)
dataset_list.append(dataset_chunk_6)

In [4]:
# chunk gluing
dataset = {}

dataset['image_list'] = dataset_list[0]['image_list']
dataset['features'] = dataset_list[0]['features']


for i in range(1, len(dataset_list)):
    dataset['image_list']+=dataset_list[i]['image_list']
    dataset['features'] = np.concatenate((dataset['features'], dataset_list[i]['features']))

In [5]:
print (len(dataset['features']))

30607


In [6]:
# reduce to 1000 features
pca = decomposition.PCA(n_components=1000, svd_solver='full')
pca.fit(dataset['features'])

PCA(copy=True, iterated_power='auto', n_components=1000, random_state=None,
  svd_solver='full', tol=0.0, whiten=False)

In [7]:
transformed_data = pca.transform(dataset['features'])

In [8]:
import sklearn.mixture

n_clusters = 256
        
clusterer = sklearn.mixture.GaussianMixture(n_clusters)
clusterer.fit(transformed_data)

GaussianMixture(covariance_type='full', init_params='kmeans', max_iter=100,
        means_init=None, n_components=256, n_init=1, precisions_init=None,
        random_state=None, reg_covar=1e-06, tol=0.001, verbose=0,
        verbose_interval=10, warm_start=False, weights_init=None)

In [9]:
predictions = clusterer.predict(transformed_data)

In [10]:
#place results in this directory
harder_dataset_dir_path = './256_dataset_GMM/'

if not os.path.exists(harder_dataset_dir_path):
        os.makedirs(harder_dataset_dir_path)
        
for i in range(n_clusters):
    newpath = harder_dataset_dir_path + 'class_' + str(i)
    if not os.path.exists(newpath):
        os.makedirs(newpath)

for i in range(len(dataset['image_list'])):
    if i % 10000 == 0:
        print i
    # directory with images is positioned in parent directory
    path = '../' + dataset['image_list'][i]
    
    save_path = harder_dataset_dir_path + r'class_' + str(predictions[i])
    tokens = path.decode("utf-8").split('/')
    copyfile(path.decode("utf-8"), save_path + '/' + tokens[len(tokens) - 1])

0
10000
20000
30000


In [11]:
harder_dataset_dir_path = './256_dataset_GMM/'
directories = []
for file in os.listdir(harder_dataset_dir_path):
    directories.append(harder_dataset_dir_path + file)

In [12]:
feature_vectors = []
labels = []
labels_true = []

for i, class_dir in enumerate(directories):
    class_id = class_dir.split('_')[-1]
    images_in_dir = os.listdir(class_dir)
    for image_in_dir in images_in_dir:
        image_in_dir_name = image_in_dir.split('/')[-1]
        for i, image in enumerate(dataset['image_list']):
            file_name = dataset['image_list'][i].split('/')[-1]
            if file_name == image_in_dir_name:
                feature_vectors.append(dataset['features'][i])
                labels.append(class_id)
                labels_true.append(dataset['image_list'][i].split('_')[0].split('/')[-1])
                continue

In [13]:
from sklearn import metrics

In [14]:
lab_tr = [int(elem) for elem in labels_true]

In [15]:
print "ARI for this dataset is", metrics.adjusted_rand_score(lab_tr, labels)

ARI for this dataset is 0.483129090198
